In [17]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")

## Dataset Pre-processing

In [18]:
TASKS = ["climbing","jumping","running"]
LABELS = {
    "climbing" : 1,
    "jumping" : 2,
    "running" : 3,
}
SENSORS = ["Accelerometer","Gyroscope"]
dataset_path = "dataset/train/"

In [19]:
data = {}
for sensors in SENSORS:
    data[sensors] = []
for vals in TASKS:
    sub_dirs = [x for x in os.walk(dataset_path + vals)]
    datapoints = sub_dirs[0][1]
    for pts in datapoints:
        for sensors in SENSORS:
            file_path = dataset_path + vals + "/" + pts + "/" +sensors + ".csv"
            df = pd.read_csv(file_path)
            df["Label"] = LABELS[vals]
            data[sensors].append(df)

In [20]:
acc = data["Accelerometer"]
gyro = data["Gyroscope"]

In [21]:
for i in range(len(acc)):
    print(acc[i].shape,gyro[i].shape)

(3025, 6) (3034, 6)
(2428, 6) (2437, 6)
(3210, 6) (3221, 6)
(1809, 6) (1825, 6)
(2185, 6) (2203, 6)
(1747, 6) (1763, 6)
(1849, 6) (1867, 6)
(3025, 6) (3034, 6)
(2428, 6) (2437, 6)
(3210, 6) (3221, 6)


In [22]:
for i in range(len(acc)):
    length = min(acc[i].shape[0],gyro[i].shape[0])
    acc[i] = acc[i][:length]
    gyro[i] = gyro[i][:length]

In [23]:
#Removing top and last 50 values
a = 0
N = 50
for i in range(len(acc)):
    acc[i] = acc[i][N:-N]
    gyro[i] = gyro[i][N:-N]
    a = a + acc[i].shape[0]
print(a)

23916


In [24]:
dfa = pd.concat(acc)
dfg = pd.concat(gyro)

In [25]:
#dropping timestamps and milliseconds
try:
    dfa.drop(dfa[["Timestamp","Milliseconds"]],inplace=True,axis=1)
    dfg.drop(dfg[["Timestamp","Milliseconds"]],inplace=True,axis=1)
except:
    pass
dfa = dfa.reset_index(drop=True)
dfg = dfg.reset_index(drop=True)

In [26]:
dfa.columns = ["ACCX","ACCY","ACCZ","OUTPUT"]
dfg.columns = ["GYROX","GYROY","GYROZ","OUTPUT"]

In [27]:
dfa.tail(10)

,ACCX,ACCY,ACCZ,OUTPUT
23906,-2.042693,3.788111,11.212740,3
23907,-1.934959,3.898239,11.681983,3
23908,-1.867924,3.958092,12.175167,3
23909,-1.918200,3.850357,12.644411,3
23910,-1.990023,3.706711,12.929309,3
23911,-2.157610,3.462513,12.912550,3
23912,-2.581366,3.417025,12.704263,3
23913,-2.679524,3.713893,11.993216,3
23914,-2.435326,4.080191,11.181617,3
23915,-2.272527,4.293266,10.729132,3


In [28]:
df = ""
if(dfg['OUTPUT'].equals(dfa["OUTPUT"])):
    print("Merging Tables")
    dfa.drop(dfa[["OUTPUT"]],axis=1,inplace=True)
    df = pd.concat([dfa,dfg],axis=1)
else:
    print("Error in merging tables")
df.head(100)

Merging Tables


,ACCX,ACCY,ACCZ,GYROX,GYROY,GYROZ,OUTPUT
0,4.780488,3.426602,5.378321,0.502612,-0.841168,1.029681,1
1,5.252126,3.907815,4.937807,-0.060605,-0.869268,0.892848,1
2,5.546600,4.197502,5.086241,-0.292734,-0.903476,0.842757,1
3,5.173121,3.922180,5.493238,-0.743552,-1.031758,0.767010,1
4,4.349551,3.206344,7.164319,-1.178488,-1.492350,0.832983,1
...,...,...,...,...,...,...,...
95,4.526714,3.270985,6.103733,-0.775317,0.076351,-0.597662,1
96,4.603325,3.503213,6.867450,-0.686131,-0.225416,-0.386303,1
97,4.770912,3.790505,7.683838,-0.548075,-0.430667,-0.236030,1
98,5.304796,4.247777,8.658236,-0.372146,-0.601709,-0.112636,1


In [30]:
df.shape

(23916, 7)

In [84]:
def build_model(x_train, y_train):
    
    n_time_steps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
    
    verbose, epochs, batch_size = 0, 70, 16
    
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')

    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
